In [1]:
%matplotlib inline

In [2]:
#-------------------------------------------------------------------------------------------------------------------------------
# By Alexandra Lee (July 2018) 
#
# Generate input files
#
# Dataset: Pseudomonas aeruginosa gene expression compendium referenced in https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5069748/
# 
# Use map_file to group samples into phenotype groups (condition A and B) based on experimental design annotations
# Example: control vs treatment with antibiotics
# 
# Then group samples into training and test sets
#
# Generate offset vector using gene expression data in the original space (train_offset_original):
# average gene expression for condition A - average gene expression for condition B using all genes/dimensions
#-------------------------------------------------------------------------------------------------------------------------------
import os
import pandas as pd
import numpy as np
from scipy.stats import variation
import seaborn as sns
import matplotlib.pyplot as plt

randomState = 123
from numpy.random import seed
seed(randomState)

In [3]:
# load arguments
data_file = os.path.join(os.path.dirname(os.getcwd()), "data", "all-pseudomonas-gene-normalized.zip")  # repo file is zipped
map_file = os.path.join(os.path.dirname(os.getcwd()), "metadata", "mapping_oxy.txt")

# output
train_max_file = os.path.join(os.path.dirname(os.getcwd()), "data", "oxygen_level", "train_maxO2.txt")
train_min_file = os.path.join(os.path.dirname(os.getcwd()), "data", "oxygen_level", "train_minO2.txt")
train_input_file = os.path.join(os.path.dirname(os.getcwd()), "data", "oxygen_level", "train_model_input.txt.xz")
original_offset_file = os.path.join(os.path.dirname(os.getcwd()), "data", "oxygen_level", "train_offset_original.txt")

In [4]:
# read in data
data = pd.read_table(data_file, header = 0, sep = '\t', index_col = 0, compression='zip')
X = data.transpose()
X.head(5)

Gene_symbol,PA0001,PA0002,PA0003,PA0004,PA0005,PA0006,PA0007,PA0008,PA0009,PA0010,...,PA5561,PA5562,PA5563,PA5564,PA5565,PA5566,PA5567,PA5568,PA5569,PA5570
0.1_12hr_CSV86(Pae_G1a).CEL,0.472897,0.396658,0.253776,0.000000,0.175640,0.554385,0.411370,0.382222,0.310144,0.642522,...,0.358597,0.390048,0.457406,0.684082,0.338351,0.608325,0.643496,0.276075,0.112773,0.145170
0.1_2hr_CSV86(Pae_G1a).CEL,0.262346,0.086216,0.359853,0.439214,0.269749,0.768433,0.212505,0.062043,0.567695,0.467073,...,0.358504,0.414206,0.389879,0.477693,0.000000,0.479385,0.154471,0.140891,0.167505,0.157060
0.1_6hr_CSV86(Pae_G1a).CEL,0.473658,0.244862,0.330750,0.097697,0.387226,0.328319,0.228820,0.330039,0.318081,0.512864,...,0.180744,0.380741,0.173501,0.251571,0.182793,0.528301,0.504985,0.499782,0.061106,0.365612
0.1_7hr_CSV86(Pae_G1a).CEL,0.439273,0.343402,0.192698,0.274677,0.628979,0.553796,0.431391,0.363480,0.385721,0.094584,...,0.346837,0.153927,0.067349,0.319723,0.282442,0.490655,0.531415,0.153880,0.132333,0.260087
0.1_9hr_CSV86(Pae_G1a).CEL,0.220827,0.145525,0.437803,0.293201,0.635120,0.462893,0.488733,0.309584,0.318646,0.591914,...,0.237726,0.301945,0.070222,0.513605,0.114277,0.360259,0.386868,0.223995,0.105343,0.102088


In [5]:
# read in metadata file containing grouping of each sample into training/test and phenotypic group
grp = pd.read_table(map_file, header = 0, sep = '\t', index_col = None)
grp

,Experiment ID,Sample ID,Phenotype,Group
0,E-GEOD-52445,GSM1267086_HZI1981_G1a.CEL,maxO2,Train
1,E-GEOD-52445,GSM1267087_HZI1950_Pae_G1a.CEL,t5,Test
2,E-GEOD-52445,GSM1267088_HZI1951_Pae_G1a.CEL,t10,Test
3,E-GEOD-52445,GSM1267089_HZI1952_Pae_G1a.CEL,t15,Test
4,E-GEOD-52445,GSM1267090_HZI1953a_Pae_G1a.CEL,t20,Test
5,E-GEOD-52445,GSM1267091_HZI1954_Pae_G1a.CEL,t25,Test
6,E-GEOD-52445,GSM1267092_HZI1955_Pae_G1a.CEL,t30,Test
7,E-GEOD-52445,GSM1267093_HZI1956_Pae_G1a.CEL,t35,Test
8,E-GEOD-52445,GSM1267094_HZI1958_Pae_G1a.CEL,t40,Test
9,E-GEOD-52445,GSM1267095_HZI1959_Pae_G1a.CEL,t50,Test


In [6]:
# Group samples into training and test sets
# Training: min and max levels of O2
# Test: all intermediate levels

maxO2 = pd.DataFrame()
minO2 = pd.DataFrame()
intermediate = pd.DataFrame()

for index, row in grp.iterrows():
    if row['Group'] == 'Train':
        if row['Phenotype'] == "maxO2":
            sample = str(row['Sample ID'])
            maxO2 = maxO2.append(X[X.index.str.contains(sample, regex=False)])
        else:
            sample = str(row['Sample ID'])
            minO2 = minO2.append(X[X.index.str.contains(sample, regex=False)])
    if row['Group'] == 'Test':
        sample = str(row['Sample ID'])
        intermediate = intermediate.append(X[X.index.str.contains(sample, regex=False)])

#maxO2
#minO2
intermediate

Gene_symbol,PA0001,PA0002,PA0003,PA0004,PA0005,PA0006,PA0007,PA0008,PA0009,PA0010,...,PA5561,PA5562,PA5563,PA5564,PA5565,PA5566,PA5567,PA5568,PA5569,PA5570
GSM1267087_HZI1950_Pae_G1a.CEL,0.691875,0.773698,0.310379,0.728245,0.362461,0.174097,0.930514,0.687953,0.524678,0.190374,...,0.235338,0.536374,0.572380,0.634021,0.744842,0.132808,0.369619,0.557571,0.679192,0.696873
GSM1267088_HZI1951_Pae_G1a.CEL,0.637564,0.769579,0.370289,0.728375,0.398652,0.188218,0.872109,0.710807,0.597722,0.180090,...,0.339106,0.505761,0.622343,0.724826,0.812770,0.201955,0.435832,0.704239,0.817787,0.768841
GSM1267089_HZI1952_Pae_G1a.CEL,0.606916,0.760584,0.302940,0.764910,0.418426,0.271575,0.914675,0.729262,0.611175,0.165047,...,0.386966,0.447978,0.562056,0.646066,0.815421,0.144954,0.321098,0.648080,0.793622,0.774694
GSM1267090_HZI1953a_Pae_G1a.CEL,0.602017,0.725574,0.304382,0.665943,0.464325,0.290835,0.756532,0.665519,0.580099,0.162599,...,0.467639,0.506814,0.568582,0.692956,0.781432,0.181212,0.422187,0.615707,0.795466,0.743028
GSM1267091_HZI1954_Pae_G1a.CEL,0.634527,0.665532,0.318573,0.601329,0.434796,0.319123,0.684439,0.554885,0.542199,0.177594,...,0.467207,0.515335,0.630096,0.756791,0.839536,0.162397,0.447986,0.564840,0.762237,0.693278
GSM1267092_HZI1955_Pae_G1a.CEL,0.806269,0.725206,0.551114,0.705956,0.636264,0.445028,0.423975,0.785459,0.849989,0.202822,...,0.686019,0.617961,0.725759,0.914412,0.957394,0.101822,0.560775,0.791550,0.964959,0.876074
GSM1267093_HZI1956_Pae_G1a.CEL,0.608048,0.641269,0.356136,0.569670,0.409309,0.352084,0.538632,0.527709,0.520000,0.171095,...,0.489666,0.567968,0.643942,0.766778,0.845690,0.195684,0.505992,0.576437,0.780177,0.730734
GSM1267094_HZI1958_Pae_G1a.CEL,0.622009,0.641335,0.323880,0.558421,0.387747,0.332984,0.475819,0.587416,0.534537,0.155466,...,0.471634,0.560419,0.648139,0.779127,0.864147,0.279475,0.564182,0.622324,0.830900,0.743692
GSM1267095_HZI1959_Pae_G1a.CEL,0.545001,0.664846,0.303048,0.483814,0.329816,0.329092,0.358711,0.479774,0.406343,0.173398,...,0.422897,0.552958,0.591770,0.685880,0.756882,0.374701,0.631114,0.566310,0.714278,0.668839
GSM1267096_HZI1960a_Pae_G1a.CEL,0.692916,0.722953,0.399689,0.626520,0.398254,0.374881,0.375212,0.600820,0.637316,0.270108,...,0.595453,0.599133,0.679371,0.777698,0.864046,0.158510,0.560554,0.596654,0.787574,0.738632


In [7]:
# Create input holding out test test (intermediate time points)
input_holdout = X.drop(intermediate.index)

input_holdout.shape
#X.shape

(1179, 5549)

In [8]:
# Generate offset using average gene expression in original dataset
train_offset_original = minO2.values - maxO2.values
train_offset_original = pd.DataFrame(train_offset_original, columns = minO2.columns)
train_offset_original

Gene_symbol,PA0001,PA0002,PA0003,PA0004,PA0005,PA0006,PA0007,PA0008,PA0009,PA0010,...,PA5561,PA5562,PA5563,PA5564,PA5565,PA5566,PA5567,PA5568,PA5569,PA5570
0,0.043402,0.003533,0.01119,0.028326,0.060555,0.007471,-0.595833,0.119227,0.074099,0.026433,...,0.254963,0.078187,0.058836,0.165388,0.036142,0.082788,0.0857,0.197216,0.137356,0.099913


In [9]:
# Output training and test sets

# training data
maxO2.to_csv(train_max_file, sep='\t')
minO2.to_csv(train_min_file, sep='\t')
input_holdout.to_csv(train_input_file, sep='\t', compression='xz')

# test data
for index, row in grp.iterrows():
    if row['Group'] == 'Test':
        sample = str(row['Sample ID'])
        df = pd.DataFrame(intermediate.loc[sample]).transpose()
        df.to_csv(os.path.join(os.path.dirname(os.getcwd()), "data", "oxygen_level", "test_"+row['Phenotype']+".txt"), sep='\t')

# original offset
train_offset_original.to_csv(original_offset_file, sep='\t')
